In [1]:
import pandas as pd
from pandas import read_csv as read
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_original = read('m_creation/outputs/general/test_general.csv')
df_classification = read('m_creation-classification/outputs/test_general.csv')
df_limited = read('m_creation-general_limited/outputs/test_general.csv')

df_original['type'] = "original"
df_original = df_original.rename(columns={"Training Subjects": "test_set"})
df_limited['type'] = "limited"

df = pd.concat([df_original, df_limited])
df = df[df['Model'] != "Least Angle Regression"]

In [3]:
subject_type = []

subjects = set()
for subjects_tuple in df['test_set']:
    subjects.update(eval(subjects_tuple))

for t in ["original", "limited"]:
    for s in subjects:
        subject_type.append(
            (s, t)
        )

In [4]:
# Create the dataframe with an additional 'Type' column in the index
columns = ['Best Model', 'Min MAE', 'Median MAE', 'Max MAE', 'Other Participants']
index = pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=['Subject', 'Type'])
df_summary_general = pd.DataFrame(columns=columns, index=index)

# Loop through each unique combination of subject and type
for (subject, type_) in subject_type:
    # Filter the dataframe for rows where the current subject and type match
    subject_df = df[df['test_set'].apply(lambda x: subject in eval(x))]
    subject_type_df = subject_df[subject_df['type'] == type_]

    # print("S: " + str(subject) + ", T: " + type_)
    # print(subject_type_df.test_set.unique())
    # print(subject_type_df.type.unique())

    if len(subject_type_df.index) != 0:
        # Find the model with the minimum MAE for the current combination of subject and type
        best_model_row = subject_type_df.loc[subject_type_df['MAE'].idxmin()]
        best_model = best_model_row['Model']

        # Find the min MAE
        min_mae = best_model_row['MAE']

        # Find the median MAE
        median_mae = subject_type_df['MAE'].median()

        # Find the max MAE
        max_mae = subject_type_df.loc[subject_type_df['MAE'].idxmax()]['MAE']

        # Find the 'Other Participants' in the best model scenario
        best_training_subjects = eval(best_model_row['test_set'])
        other_participants = tuple(set(best_training_subjects) - {subject})

        # Append to the dataframe for the current subject and type
        df_summary_general.loc[(subject, type_), :] = [best_model, min_mae, median_mae, max_mae, other_participants]

# Sort the index of the dataframe
# df_summary_general.sort_index(inplace=True)

c:\Users\Build\Documents\VSCode\shoulder_fatigue_modeling\venv\lib\site-packages\numpy\core\fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
c:\Users\Build\Documents\VSCode\shoulder_fatigue_modeling\venv\lib\site-packages\numpy\core\fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
c:\Users\Build\Documents\VSCode\shoulder_fatigue_modeling\venv\lib\site-packages\numpy\core\fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (wh

In [5]:
df_original

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,test_set,type
0,K Neighbors Regressor,0.2900,0.2832,0.5322,0.9491,0.1965,0.1167,"(1, 2, 3)",original
1,Random Forest Regressor,0.7996,1.3885,1.1783,0.7504,0.4134,0.2486,"(1, 2, 3)",original
2,Decision Tree Regressor,0.7978,2.3997,1.5491,0.5686,0.5187,0.2786,"(1, 2, 3)",original
3,Light Gradient Boosting Machine,0.9219,1.6342,1.2783,0.7062,0.4405,0.3217,"(1, 2, 3)",original
4,Extra Trees Regressor,0.9356,1.6844,1.2979,0.6972,0.4452,0.3114,"(1, 2, 3)",original
...,...,...,...,...,...,...,...,...,...
6547,Lasso Least Angle Regression,1.8727,5.2081,2.2821,0.1288,0.7078,0.7023,"(15, 16, 17)",original
6548,Huber Regressor,1.9512,5.9623,2.4418,0.0026,0.7154,0.6375,"(15, 16, 17)",original
6549,Dummy Regressor,2.0538,5.9850,2.4464,-0.0012,0.7544,0.7685,"(15, 16, 17)",original
6550,Passive Aggressive Regressor,2.6564,12.8235,3.5810,-1.1451,1.2131,0.9515,"(15, 16, 17)",original


In [95]:
df_summary_general = df_summary_general.reset_index()
df_summary_general.to_csv('summary_general_comparison.csv', mode='w', header=True, index=False)


In [100]:
df_classification_summary = df_classification.groupby('Model').agg({'Accuracy': 'median', 'Recall': 'median', 'Prec.': 'median', 'F1': 'median'}).reset_index()
df_classification_summary.to_csv('df_classification_summary.csv', mode='w', header=True, index=False)